In [1]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

In [2]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

# Import Data

In [ ]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

In [ ]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

# Import Data

In [7]:
dfRes = pd.read_csv('OUTPUT/MP/05-classifiers/DataV2/CLFRunCombined_SelCol.csv')

In [8]:
dfRes

,MainDataset,Title,date,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV,boruta_params
0,T81,T81_Holdout,2024-04-27 18:38:36.112744,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.845993,0.830796,0.967042,0.799135,0.685160,0.685160,0.976432,0.959327,0.792884,0.959327,Boruta_T1000_itrr500_th100_topR5
1,T81,T81_Holdout,2024-04-27 18:38:36.112744,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.849266,0.834664,0.969056,0.804437,0.694857,0.694857,0.974471,0.956449,0.797947,0.956449,Boruta_T1000_itrr500_th100_topR5
2,T81,T81_Holdout,2024-04-27 18:38:36.112744,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,0.885006,0.876654,0.971158,0.860965,0.796562,0.796562,0.956746,0.937275,0.853209,0.937275,Boruta_T1000_itrr500_th100_topR5
3,T81,T81_Holdout,2024-04-27 18:38:36.112744,RandomForestClassifier(),6.0,1000.0,42.0,NaN,NaN,0.891533,0.883766,0.972673,0.869714,0.808786,0.808786,0.958746,0.940969,0.860844,0.940969,Boruta_T1000_itrr500_th100_topR5
4,T81,T81_Holdout,2024-04-27 18:38:36.112744,RandomForestClassifier(),10.0,100.0,42.0,NaN,NaN,0.927315,0.924848,0.972825,0.917351,0.900793,0.900793,0.948903,0.934733,0.921872,0.934733,Boruta_T1000_itrr500_th100_topR5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,DT,DT_Boruta_Top,2024-04-27 18:43:02.964525,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.908760,0.904975,0.946252,0.894319,0.868851,0.868851,0.941099,0.923561,0.900165,0.923561,Boruta_T1000_itrr500_th100_topR5
488,DT,DT_Boruta_Top,2024-04-27 18:43:02.964525,"XGBClassifier(base_score=None, booster=None, c...",15.0,100.0,42.0,NaN,1.0,0.914195,0.911487,0.953871,0.902118,0.885836,0.885836,0.937138,0.919394,0.910667,0.919394,Boruta_T1000_itrr500_th100_topR5
489,DT,DT_Boruta_Top,2024-04-27 18:43:02.964525,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.910934,0.907834,0.947969,0.897835,0.878548,0.878548,0.937119,0.919194,0.906017,0.919194,Boruta_T1000_itrr500_th100_topR5
490,DT,DT_Boruta_Top,2024-04-27 18:43:02.964525,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,1.0,0.918531,0.914735,0.951493,0.905639,0.878548,0.878548,0.950922,0.936336,0.907610,0.936336,Boruta_T1000_itrr500_th100_topR5
